### Adds a certain amount of images without instruments and corresponding empty masks to the training set

In [ ]:
import sys
import os, os.path
from shutil import copyfile
import random
from PIL import Image

In [ ]:
all_images = r"E:\Seafile\WFT_SS2021_Thema 2\Uploads\images_all"
training_images = r"E:\Seafile\WFT_SS2021_Thema 2\Uploads\images_training"
all_training_masks = r"E:\Seafile\WFT_SS2021_Thema 2\Uploads\masks"
json_dir_excluding_rectangles = r"E:\Seafile\WFT_SS2021_Thema 2\Uploads\json_files_excluding_rectangles"

In [ ]:
# Decide how many empty masks you want to include, with respect to the total of masks in that session
percentage_of_masks = 0.2

# Reference the Sessions and Subsessions, in case you want to add masks individually 
ses_list = ["01","02","03","04","05","06","07","08","09","10"]
sub_ses_list = ["02","03"]


for ses in ses_list:
    for sub_ses in sub_ses_list:
        masks_list = [i for i in os.listdir(all_training_masks) if os.path.isfile(os.path.join(all_training_masks,i)) and \
         i.startswith("aicm"+ses+"_"+sub_ses)]

        all_images_list = [i for i in os.listdir(all_images) if os.path.isfile(os.path.join(all_images,i)) and \
         i.startswith("aicm"+ses+"_"+sub_ses)]
        
        json_files_list = [i for i in os.listdir(json_dir_excluding_rectangles) if os.path.isfile(os.path.join(json_dir_excluding_rectangles,i)) and \
         i.startswith("aicm"+ses+"_"+sub_ses)]

        num_masks = len([i for i in os.listdir(all_training_masks) if os.path.isfile(os.path.join(all_training_masks,i)) and \
         i.startswith("aicm"+ses+"_"+sub_ses)])
        
        num_json_files = len([i for i in os.listdir(json_dir_excluding_rectangles) if os.path.isfile(os.path.join(json_dir_excluding_rectangles,i)) and \
         i.startswith("aicm"+ses+"_"+sub_ses)])
        
        # If there are less masks than intended (including empty masks), generate new empty masks
        if num_masks < num_masks+round(percentage_of_masks*num_json_files):

            num_empty_masks = num_masks+round(percentage_of_masks*num_json_files) - num_masks
            diff_list = list(set(all_images_list) - set(masks_list))

            # Pick random sample of images without masks, to generate empty ones
            empty_mask_list = random.sample(diff_list, num_empty_masks)
            for mask in empty_mask_list:
                copyfile(os.path.join(all_images,mask),os.path.join(training_images,mask))

                # Create a new, empty mask
                img = Image.open(os.path.join(all_images,mask))
                width, height = img.size

                img_new = Image.new('L', (width, height))
                img_new.save(os.path.join(all_training_masks,mask), "PNG")
        # If there are more masks than intended (including empty masks), delete certain amount of empty masks
        elif num_masks > num_masks+round(percentage_of_masks*num_json_files):
            num_empty_masks = num_masks - num_masks+round(percentage_of_masks*num_json_files)
            diff_list = list(set(masks_list) - set(json_files_list))

            # Pick random sample of empty masks to delete 
            empty_mask_list = random.sample(diff_list, num_empty_masks)
            for mask in empty_mask_list:
                os.remove(os.path.join(training_images,mask))
                os.remove(os.path.join(all_training_masks,mask))